In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn
import warnings
warnings.filterwarnings('ignore')

In [23]:
df = pd.read_csv('S&PDataset_final.csv')
df.shape

(11553, 26)

In [24]:
df['conm'].nunique()

497

In [25]:
df.isnull().sum()

GVKEY          0
datadate       0
fyear          0
indfmt         0
consol         0
popsrc         0
datafmt        0
conm           0
curcd          0
aco          428
act         2081
aqc         1079
at             0
capx         163
ceq            0
dt          1555
ebit           0
icapt          2
lct         2068
ni             0
revt           0
seq            0
xrd         4930
xrdp        4965
costat         0
sic            0
dtype: int64

In [26]:
# Counting total financial companies

rows_with_sic_starting_6 = df[df['sic'].astype(str).str.startswith('6')]
grouped_by_gvkey = rows_with_sic_starting_6.groupby('GVKEY')
unique_gvkeys = grouped_by_gvkey.groups.keys()
len(unique_gvkeys)


96

In [27]:
# Counting financial companies that have incomplete data

filtered_df = df[~df['fyear'].between(1999, 2022)]
filtered_df_starting_6 = filtered_df[filtered_df['sic'].astype(str).str.startswith('6')]
grouped_by_gvkey = filtered_df_starting_6.groupby('GVKEY')
unique_gvkeys_starting_6 = grouped_by_gvkey.groups.keys()
len(unique_gvkeys_starting_6)

95

In [28]:
# Dropping all companies that have incomplete data and financial companies

df_within_year_range = df[df['fyear'].between(1999, 2022)]
df_no_sic_starting_6 = df_within_year_range[~df_within_year_range['sic'].astype(str).str.startswith('6')]

In [29]:
print("Shape: ", df_no_sic_starting_6.shape)
print("Total remaining companies: ", df_no_sic_starting_6['conm'].nunique())

Shape:  (8607, 26)
Total remaining companies:  399


In [30]:
df_no_sic_starting_6.isnull().sum()

GVKEY          0
datadate       0
fyear          0
indfmt         0
consol         0
popsrc         0
datafmt        0
conm           0
curcd          0
aco            0
act          272
aqc          796
at             0
capx          27
ceq            0
dt           776
ebit           0
icapt          0
lct          260
ni             0
revt           0
seq            0
xrd         3024
xrdp        3051
costat         0
sic            0
dtype: int64

In [31]:
# Counting the unique companies that have missing values for total current assets and liabilities

grouped_by_gvkey = df_no_sic_starting_6.groupby('GVKEY')

missing_act = grouped_by_gvkey['act'].apply(lambda x: x.isnull().sum())
missing_lct = grouped_by_gvkey['lct'].apply(lambda x: x.isnull().sum())

missing_companies = []

for gvkey, count in missing_act.items():
    if count != 0 and count > 1:
        missing_companies.append(gvkey)

print(("{total} companies with missing act and lct values: {list}").format(total = len(missing_companies), list = missing_companies))


14 companies with missing act and lct values: [2176, 3835, 4839, 5047, 5073, 6669, 8253, 8823, 10519, 11456, 12459, 12635, 25340, 66065]


In [32]:
df_no_sic_starting_6['GVKEY'].nunique()

399

In [33]:
# Dropping companies with no act or lct values
df_final = df_no_sic_starting_6[~df_no_sic_starting_6['GVKEY'].isin(missing_companies)]
df_final['GVKEY'].nunique()

385

In [34]:
# Substituing null values for aqcuisitions to 0

df_final['aqc'].fillna(0, inplace=True)

In [35]:
# Dropping 'total debt'

df_final = df_final.drop(columns=['dt'])
df_final.columns

Index(['GVKEY', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt',
       'conm', 'curcd', 'aco', 'act', 'aqc', 'at', 'capx', 'ceq', 'ebit',
       'icapt', 'lct', 'ni', 'revt', 'seq', 'xrd', 'xrdp', 'costat', 'sic'],
      dtype='object')

In [36]:
df_final.isnull().sum()

GVKEY          0
datadate       0
fyear          0
indfmt         0
consol         0
popsrc         0
datafmt        0
conm           0
curcd          0
aco            0
act            1
aqc            0
at             0
capx          24
ceq            0
ebit           0
icapt          0
lct            1
ni             0
revt           0
seq            0
xrd         2868
xrdp        2895
costat         0
sic            0
dtype: int64

In [37]:
# df_final.to_csv('final_dataset.csv')